# Crystal relaxation and elastic constants

This Notebook follows the workflow for identifying and analyzing bulk crystal structures as is currently being done with the Interatomic Potentials Repository.

**Quick Notes:**

- All input scripts take key-value pairs where the "key"s correspond to the calculation's input parameters.

- Multiple calculations are prepared by specifying multiple values for the same key (on separate lines).

- The special "buildcombos" key accesses predefined functions for generating lists of input parameter values for certain sets of keys.  See documentation for more details.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)

# http://www.numpy.org/
import numpy as np

from IPython.core.display import display, HTML

# https://pandas.pydata.org/
import pandas as pd

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.2


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('master')
database.build_refs()

## 1. E_vs_r_scan calculation

This calculation performs a cohesive energy versus interatomic spacing scan using ideal atomic positions and b/a and c/a ratios to identify possible low energy crystal structures.

In [3]:
calculation = iprPy.load_calculation('E_vs_r_scan')
run_directory = iprPy.load_run_directory('master_1')

### Show calculation's allowed keys

These are the keys that the input script will accept.

In [4]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'minimum_r', 'maximum_r', 'number_of_steps_r']


### Write input script

In [14]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on prototype records
buildcombos                 crystalprototype load_file prototype

# Specify prototype buildcombos limiters (only build for potential listed)
prototype_potential_name    2015--Choudhary-K--Al--LAMMPS--ipr1

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                  
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
minimum_r                   0.5
maximum_r                   6.0
number_of_steps_r           276  
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [15]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 10 of style calculation_E_vs_r_scan
 - 0 are complete
 - 10 still to run
 - 0 issued errors


### Run calculations

In [17]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 12960
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 10 of style calculation_E_vs_r_scan
 - 10 are complete
 - 0 still to run
 - 0 issued errors


## 2. relax_box calculation

This calculation statically relaxes a given system by only adjusting the box dimensions to zero pressure without any internal relaxations, i.e. all atoms retain box-relative positions.

In [19]:
calculation = iprPy.load_calculation('relax_box')
run_directory = iprPy.load_run_directory('master_1')

### Show calculation's allowed keys

In [20]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'pressure_xx', 'pressure_yy', 'pressure_zz', 'strainrange']


### Write input script

In [21]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference

# Specify reference buildcombos limiters (only build for potential listed)
reference_potential_name    2015--Choudhary-K--Al--LAMMPS--ipr1

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent

# Specify parent buildcombos terms (parent record's style and the load_key to access)
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                    
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
strainrange                 1e-6
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [22]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 32 of style calculation_relax_box
 - 0 are complete
 - 32 still to run
 - 0 issued errors


### Run calculations

In [23]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 12960
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 32 of style calculation_relax_box
 - 28 are complete
 - 0 still to run
 - 4 issued errors


In [24]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
  File "calc_relax_box.py", line 458, in <module>
    main(*sys.argv[1:])
  File "calc_relax_box.py", line 54, in main
    strainrange = input_dict[\'strainrange\'])
  File "calc_relax_box.py", line 145, in relax_box
    strainrange=strainrange, cycle=cycle)
  File "calc_relax_box.py", line 346, in calc_cij
    C = am.ElasticConstants(Cij=cij)
  File "c:\\users\\lmh1\\documents\\python-packages\\atomman\\atomman\\core\\ElasticConstants.py", line 59, in __init__
    self.Cij = kwargs[\'Cij\']
  File "c:\\users\\lmh1\\documents\\python-packages\\atomman\\atomman\\core\\ElasticConstants.py", line 115, in Cij
    assert value.max() > 0.0, \'Cij values not valid\'
AssertionError: Cij values not valid
'
b'Traceback (most recent call last):
  File "calc_relax_box.py", line 458, in <module>
    main(*sys.argv[1:])
  File "calc_relax_box.py", line 54, in main
    strainrange = input_dict[\'strainrange\'])
  File "calc_relax_box.py", line 145, in relax_box
  

## 3. relax_dynamic calculation

This calculation dymamically relaxes a given system for a specified number of MD integrations at a specified temperature, pressure, etc.  Here, we are only doing 0 K relaxations.

In [25]:
calculation = iprPy.load_calculation('relax_dynamic')
run_directory = iprPy.load_run_directory('master_1')

### Write input script

In [30]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 c:\\Program Files\\MPICH2\\bin\\mpiexec -localonly 2

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference

# Specify reference buildcombos limiters (only build for potential listed)
reference_potential_name    2015--Choudhary-K--Al--LAMMPS--ipr1

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent

# Specify parent buildcombos terms (parent record's style and the load_key to access)
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw       
atomshift                   
sizemults                   10 10 10

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
temperature                 0.0
pressure_xx                 
pressure_yy                 
pressure_zz                 
pressure_xy                 
pressure_xz                 
pressure_yz                 
integrator                  nph+l
thermosteps                 1000
dumpsteps                   
runsteps                    10000
equilsteps                  0
randomseed                  
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [31]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 32 of style calculation_relax_dynamic
 - 0 are complete
 - 32 still to run
 - 0 issued errors


### Run calculations

In [ ]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 12960


In [29]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
un.py", line 110, in run
    output = sp.check_output(mpi_command + lammps_command + extra + [\'-in\'] + script_name)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command \'[\'mpiexec\', \'-localonly\', \'2\', \'lmp_mpi\', \'-in\', \'full_relax.in\']\' returned non-zero exit status 4294967295.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "calc_relax_dynamic.py", line 505, in <module>
    main(*sys.argv[1:])
  File "calc_relax_dynamic.py", line 59, in main
    randomseed = input_dict[\'randomseed\'])
  File "calc_relax_dynamic.py", line 223, in relax_dynamic
    output = lmp.run(lammps_command, lammps_script, mpi_command)
un.py", line 11

## 4. relax_static calculation

This calculation statically relaxes a given system using energy minimizations combined with box dimension relaxations.  Here, we pass in results from both the E_vs_r_scan calculation and the relax_dynamic calculation.

In [11]:
calculation = iprPy.load_calculation('relax_static')
run_directory = iprPy.load_run_directory('master_1')

### Write input script

In [12]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information based on reference structures
buildcombos                 atomicreference load_file reference

# Specify reference buildcombos limiters (only build for potential listed)
reference_potential_name    2015--Choudhary-K--Al--LAMMPS--ipr1

# Build load information from E_vs_r_scan results
buildcombos                 atomicparent load_file parent

# Specify parent buildcombos terms (parent record's style and the load_key to access)
parent_record               calculation_E_vs_r_scan              
parent_load_key             minimum-atomic-system

# Build load information from relax_dynamic results
buildcombos                 atomicarchive load_file archive

# Specify archive parent buildcombos terms (parent record's style and the load_key to access)
archive_record              calculation_relax_dynamic
archive_load_key            final-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   1 1 1

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             0.0
forcetolerance              1e-10 eV/angstrom
maxiterations               10000
maxevaluations              100000
maxatommotion               0.01 angstrom
maxcycles                   100
cycletolerance              1e-10
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [13]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 64 of style calculation_relax_static
 - 0 are complete
 - 64 still to run
 - 0 issued errors


### Run calculations

In [14]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 15832
Didn't find an open simulation
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 64 of style calculation_relax_static
 - 54 are complete
 - 1 still to run
 - 9 issued errors


In [15]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\Anaconda3\\lib\\shutil.py", line 544, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 2] The system cannot find the file specified: \'10000.dump\' -> \'relax_static-0.dump\'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "calc_relax_static.py", line 380, in <module>
    main(*sys.argv[1:])
  File "calc_relax_static.py", line 62, in main
    ctol = input_dict[\'cycletolerance\'])
  File "calc_relax_static.py", line 231, in relax_static
    shutil.move(last_dump_file, renamed_dump_file)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\Anaconda3\\lib\\shutil.py", line 558, in move
    copy_function(src, real_dst)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\Anaconda3\\lib\\shutil.py", line 257, in copy2
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\

## 5. crystal_space_group calculation

This calculation analyzes the space group of a given system.  Here, this is used to determine if the bulk system's structure has transformed.

In [16]:
calculation = iprPy.load_calculation('crystal_space_group')
run_directory = iprPy.load_run_directory('master_1')

In [17]:
print(calculation.allkeys)

['length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'symmetryprecision', 'primitivecell', 'idealcell']


### Write input script

In [18]:
input_script = """

# Build load information based on prototype records
buildcombos                 crystalprototype load_file

# Build load information based on reference structures
buildcombos                 atomicreference load_file ref

# Specify reference buildcombos limiters (only build for element sets listed)
ref_elements                Al

# Build load information from relax_static results
buildcombos                 atomicarchive load_file relax_static

# Specify archive parent buildcombos terms (parent record's style and the load_key to access)
relax_static_record         calculation_relax_static
relax_static_load_key       final-system

# Build load information from relax_box results
buildcombos                 atomicarchive load_file relax_box

# Specify archive parent buildcombos terms (parent record's style and the load_key to access)
relax_box_record            calculation_relax_box
relax_box_load_key          final-system

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
symmetryprecision           
primitivecell               
idealcell                   
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

### Prepare calculations

In [19]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 4912 of style calculation_crystal_space_group
 - 4857 are complete
 - 55 still to run
 - 0 issued errors


### Run calculations

In [20]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 15832
Didn't find an open simulation
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\master :
- 4912 of style calculation_crystal_space_group
 - 4912 are complete
 - 0 still to run
 - 0 issued errors


In [29]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

## 6. Analyze

This code analyzes the results of the previous calculations to identify the systems that did not transform upon relaxation.

### Get parent records

In [30]:
relax_box_df = database.get_records_df(style='calculation_relax_box', full=True, flat=True)
relax_static_df = database.get_records_df(style='calculation_relax_static', full=True, flat=True)
relax_dynamic_df = database.get_records_df(style='calculation_relax_dynamic', full=True, flat=True)
crystal_space_group_df = database.get_records_df(style='calculation_crystal_space_group', full=True, flat=True) 

### Split into reference, prototype and results

In [31]:
reference_df = crystal_space_group_df[(crystal_space_group_df.family+'.poscar'==crystal_space_group_df.load_file)]
prototype_df = crystal_space_group_df[(crystal_space_group_df.family+'.json'==crystal_space_group_df.load_file)]
reference_index = reference_df.index
prototype_index = prototype_df.index
relaxed_index = crystal_space_group_df.index.difference(reference_index.union(prototype_index))
relaxed_df = crystal_space_group_df.iloc[relaxed_index]

reference_df = reference_df.reset_index()
prototype_df = prototype_df.reset_index()
relaxed_df = relaxed_df.reset_index()

### Indentify structures that transformed

In [32]:
match_df = pd.read_csv('reference_prototype_match.csv')

In [33]:
results_df = []
for series in relaxed_df.itertuples():
    results_dict = {}
    results_dict['key'] = series.key
    results_dict['family'] = series.family
    results_dict['a (Å)'] = series.a
    results_dict['b (Å)'] = series.b
    results_dict['c (Å)'] = series.c
    results_dict['alpha'] = series.alpha
    results_dict['beta'] = series.beta
    results_dict['gamma'] = series.gamma
    
    try:
        family_series = reference_df[reference_df.family == series.family].iloc[0]
    except:
        family_series = prototype_df[prototype_df.family == series.family].iloc[0]
        results_dict['prototype'] = series.family
    else:
        prototype = match_df[match_df.reference==series.family].prototype.values[0]
        if prototype != 'multiple':
            results_dict['prototype'] = prototype
    
    results_dict['transformed'] = not (family_series.spacegroup_number == series.spacegroup_number
                                       and family_series.pearson_symbol == series.pearson_symbol)
    
    for parent in database.get_parent_records(name=series.key):
        parent_dict = parent.todict()
        if parent_dict['key'] in relax_box_df.key.tolist():
            results_dict['method'] = 'box'
            results_dict['E_cohesive (eV)'] = parent_dict['E_cohesive']
            results_dict['potential_LAMMPS_id'] = parent_dict['potential_LAMMPS_id']
            continue
        elif parent_dict['key'] in relax_dynamic_df.key.tolist():
            results_dict['method'] = 'dynamic'
        elif parent_dict['key'] in relax_static_df.key.tolist():
            if 'relaxation' not in results_dict:
                results_dict['method'] = 'static'
            results_dict['E_cohesive (eV)'] = parent_dict['E_cohesive']
            results_dict['potential_LAMMPS_id'] = parent_dict['potential_LAMMPS_id']
    
    results_df.append(results_dict)
results_df = pd.DataFrame(results_df).sort_values('E_cohesive (eV)')

### Filter out transformed and duplicate structures

In [34]:
filtered_df = results_df[results_df.transformed == False].reset_index()

skipindex = set()
nresults = len(filtered_df)
for i in range(nresults):
    iseries = filtered_df.iloc[i]
    for j in range(i+1, nresults):
        jseries = filtered_df.iloc[j]
        if (np.isclose(iseries['E_cohesive (eV)'], jseries['E_cohesive (eV)'], rtol=0.0, atol=0.001)
            and np.isclose(iseries['a (Å)'], jseries['a (Å)'], rtol=0.0, atol=0.001)
            and np.isclose(iseries['b (Å)'], jseries['b (Å)'], rtol=0.0, atol=0.001)
            and np.isclose(iseries['c (Å)'], jseries['c (Å)'], rtol=0.0, atol=0.001)):
            
            if iseries.method == 'dynamic' or iseries.method == jseries.method:
                skipindex.add(j)
            elif jseries.method == 'dynamic':
                skipindex.add(i)
            elif iseries.method == 'static':
                skipindex.add(j)
            elif jseries.method == 'static':
                skipindex.add(i)
            else:
                raise ValueError('oops!')
            
filtered_df = filtered_df.iloc[filtered_df.index.difference(skipindex)]

In [35]:
show_df = filtered_df[filtered_df.potential_LAMMPS_id == '2006--Williams-P-L--Ag--LAMMPS--ipr1']
show_df = show_df[['prototype', 'method', 'E_cohesive (eV)', 'a (Å)', 'b (Å)', 'c (Å)', 'alpha', 'beta', 'gamma']]
display(HTML(show_df.to_html(index=False)))

## 7. elastic_constants_static calculation

In [36]:
calculation = iprPy.load_calculation('elastic_constants_static')
run_directory = iprPy.load_run_directory('demo_1')

In [37]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'strainrange', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


### Write input script

In [38]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information from relax_dynamic results
buildcombos                 atomicparent load_file parent
parent_record               calculation_crystal_space_group
parent_load_key             unit-cell-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   3 3 3

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
strainrange                  1e-5
strainrange                  1e-6
strainrange                  1e-7
strainrange                  1e-8
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

In [39]:
# Parse input script
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

### Add parent_name list using filtered results

In [40]:
input_dict['parent_name'] = filtered_df.key.tolist()

In [41]:
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 60 of style calculation_elastic_constants_static
 - 0 are complete
 - 60 still to run
 - 0 issued errors


In [42]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 6324
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 60 of style calculation_elastic_constants_static
 - 52 are complete
 - 0 still to run
 - 8 issued errors


In [43]:
results_df = database.get_records_df(style=calculation.record_style)
for error in np.unique(results_df[results_df.status=='error'].error):
    print(error)

b'Traceback (most recent call last):
un.py", line 110, in run
    output = sp.check_output(mpi_command + lammps_command + extra + [\'-in\'] + script_name)
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "C:\\Users\\lmh1\\AppData\\Local\\Continuum\\anaconda3\\lib\\subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command \'[\'lmp_mpi\', \'-in\', \'cij.in\']\' returned non-zero exit status 1.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "calc_elastic_constants_static.py", line 282, in <module>
    main(*sys.argv[1:])
  File "calc_elastic_constants_static.py", line 55, in main
    dmax = input_dict[\'maxatommotion\'])
  File "calc_elastic_constants_static.py", line 161, in elastic_constants_static
    output = lmp.run(lammps_command, lammps_script, mpi_command)
un.py", line 116, in run
 

In [44]:
elastic_df = database.get_records_df(style=calculation.record_style)

In [45]:
set(elastic_df.family)

{'A1--Cu--fcc',
 'A15--beta-W',
 'A2--W--bcc',
 "A3'--alpha-La--double-hcp",
 'A4--C--dc',
 'Ah--alpha-Po--sc',
 'mp-989737',
 'oqmd-10125',
 'oqmd-1214680',
 'oqmd-1214769',
 'oqmd-1214858',
 'oqmd-1215571',
 'oqmd-1215927'}

In [52]:
print('strain    C11     C12     C44')
for elastic_series in elastic_df[elastic_df.family == "A1--Cu--fcc"].sort_values(['strainrange']).itertuples():
    try:
        print('%s %7.2f %7.2f %7.2f' % (elastic_series.strainrange,
                                        uc.get_in_units(elastic_series.C.Cij[0,0], 'GPa'),
                                        uc.get_in_units(elastic_series.C.Cij[0,1], 'GPa'),
                                        uc.get_in_units(elastic_series.C.Cij[3,3], 'GPa')))
    except:
        pass

strain    C11     C12     C44
1e-08  124.24   93.87   46.42
1e-07  124.24   93.87   46.42
1e-06  124.24   93.87   46.42
1e-05  124.24   93.87   46.42
